In [39]:
import re
from collections import Counter

In [2]:
with open('text.txt', 'r') as file:
    TEXT = file.read()

In [145]:
def get_words(text):
    return re.findall(r'[а-я]+', text.lower(), re.I)

In [101]:
WORDS = get_words(TEXT)
WORDS[:3]

['маленький', 'принц', 'антуан']

In [94]:
COUNTS = Counter(WORDS)

In [95]:
COUNTS.most_common(10)

[('и', 57476),
 ('в', 48831),
 ('не', 32675),
 ('на', 30950),
 ('я', 28936),
 ('что', 23876),
 ('с', 19826),
 ('как', 13116),
 ('он', 12796),
 ('а', 11883)]

In [102]:
TOTAL_WORDS = sum(COUNTS.values())

In [103]:
def pword(word):
    return COUNTS[word] / TOTAL_WORDS

In [177]:
def pwords(words):
    result = 1
    for w in words:
        result *= pword(w)
    return result

In [199]:
def split_word(word, start=0):
    return [(word[:i], word[i:]) for i in range(start, len(word) + 1)]

In [200]:
split_word('привет')

[('', 'привет'),
 ('п', 'ривет'),
 ('пр', 'ивет'),
 ('при', 'вет'),
 ('прив', 'ет'),
 ('приве', 'т'),
 ('привет', '')]

In [201]:
alph = ''.join([chr(i) for i in range(1072, 1104)])
alph

'абвгдежзийклмнопрстуфхцчшщъыьэюя'

In [202]:
goal = 'привет'

In [203]:
split_word(goal)

[('', 'привет'),
 ('п', 'ривет'),
 ('пр', 'ивет'),
 ('при', 'вет'),
 ('прив', 'ет'),
 ('приве', 'т'),
 ('привет', '')]

In [224]:
def split_word_segment(text, l = 35):
    return [(text[:i], text[i:]) for i in range(1, min(len(text) + 1, l))]

In [225]:
def segment(text):
    if not text:
        return []
    return max(([a] + segment(b) for (a,b) in split_word_segment(text)), key=pwords)

In [204]:
def variants(word):
    swap_char   = [a + b[1] + b[0] +b[2:] for (a,b) in split_word(word) if len(b) > 1]
    miss_char   = [a + c + b for (a,b) in split_word(word) for c in alph]
    incor_char = [a + c + b[1:] for (a,b) in split_word(word) for c in alph if len(b) > 1]
    ex_char     = [a + b[1:] for (a,b) in split_word(word) if len(b) > 0]
    return set(swap_char + miss_char + incor_char + ex_char)

In [252]:
def known(words):
    return {w for w in words if w in COUNTS}

def edits0(word):
    return {word}

def edits1(word):
    return variants(word)

def edits2(word):
    return set(v for w in edits1(word) for v in edits1(w))

def correct_word(word):
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  {word})
    if len(candidates) == 1 and not known(candidates):
        return ' '.join(segment(list(candidates)[0]))
    return max(candidates, key=COUNTS.get)

In [253]:
def correct_match(match):
    return correct_word(match.group())

def correct(text):
    return re.sub(r'[а-я]+', correct_match, text, flags=re.I)

In [254]:
max(WORDS, key=len)

'четырехсоттридцатидевятилетнего'

In [256]:
correct('рпивет, кк втои делка? сегодянхорошийдень')

'привет, как твои дела? с его д ян хороший день'

In [231]:
segment('приветкакдела')

['привет', 'как', 'дела']